# COGS 118B - Final Project

# Airline Sentiment Analysis

## Group members

- Sukhman Virk
- Athira Rajiv
- Neil Bajaj
- Yash Sharma
- Lucas Fasting

# Abstract 
This section should be short and clearly stated. It should be a single paragraph <200 words.  It should summarize: 
- what your goal/problem is
- what the data used represents 
- the solution/what you did
- major results you came up with (mention how results are measured) 

__NB:__ this final project form is much more report-like than the proposal and the checkpoint. Think in terms of writing a paper with bits of code in the middle to make the plots/tables

# Background

Fill in the background and discuss the kind of prior work that has gone on in this research area here. **Use inline citation** to specify which references support which statements.  You can do that through HTML footnotes (demonstrated here). I used to reccommend Markdown footnotes (google is your friend) because they are simpler but recently I have had some problems with them working for me whereas HTML ones always work so far. So use the method that works for you, but do use inline citations.

Here is an example of inline citation. After government genocide in the 20th century, real birds were replaced with surveillance drones designed to look just like birds<a name="lorenz"></a>[<sup>[1]</sup>](#lorenznote). Use a minimum of 2 or 3 citations, but we prefer more <a name="admonish"></a>[<sup>[2]</sup>](#admonishnote). You need enough citations to fully explain and back up important facts. 

Remeber you are trying to explain why someone would want to answer your question or why your hypothesis is in the form that you've stated. 

# Problem Statement

Clearly describe the problem that you are solving. Avoid ambiguous words. The problem described should be well defined and should have at least one ML-relevant potential solution. Additionally, describe the problem thoroughly such that it is clear that the problem is quantifiable (the problem can be expressed in mathematical or logical terms), measurable (the problem can be measured by some metric and clearly observed), and replicable (the problem can be reproduced and occurs more than once).

# Data

Detail how/where you obtained the data and cleaned it (if necessary)

If the data cleaning process is very long (e.g., elaborate text processing) consider describing it briefly here in text, and moving the actual clearning process to another notebook in your repo (include a link here!).  The idea behind this approach: this is a report, and if you blow up the flow of the report to include a lot of code it makes it hard to read.

Please give the following infomration for each dataset you are using
- link/reference to obtain it
- description of the size of the dataset (# of variables, # of observations)
- what an observation consists of
- what some critical variables are, how they are represented
- any special handling, transformations, cleaning, etc you have done should be demonstrated here!


# Proposed Solution

In this section, clearly describe a solution to the problem. The solution should be applicable to the project domain and appropriate for the dataset(s) or input(s) given. Provide enough detail (e.g., algorithmic description and/or theoretical properties) to convince us that your solution is applicable. Make sure to describe how the solution will be tested.  

If you know details already, describe how (e.g., library used, function calls) you plan to implement the solution in a way that is reproducible.

If it is appropriate to the problem statement, describe a benchmark model<a name="sota"></a>[<sup>[3]</sup>](#sotanote) against which your solution will be compared. 

# Evaluation Metrics

Propose at least one evaluation metric that can be used to quantify the performance of both the benchmark model and the solution model. The evaluation metric(s) you propose should be appropriate given the context of the data, the problem statement, and the intended solution. Describe how the evaluation metric(s) are derived and provide an example of their mathematical representations (if applicable). Complex evaluation metrics should be clearly defined and quantifiable (can be expressed in mathematical or logical terms).

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
from nltk.corpus import stopwords

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/svirk/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
df = pd.read_csv('Tweets.csv')

In [4]:
df

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,569587686496825344,positive,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,0,@AmericanAir thank you we got on a different f...,NaN,2015-02-22 12:01:01 -0800,NaN,NaN
14636,569587371693355008,negative,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,@AmericanAir leaving over 20 minutes Late Flig...,NaN,2015-02-22 11:59:46 -0800,Texas,NaN
14637,569587242672398336,neutral,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines to...,NaN,2015-02-22 11:59:15 -0800,"Nigeria,lagos",NaN
14638,569587188687634433,negative,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my ...",NaN,2015-02-22 11:59:02 -0800,New Jersey,Eastern Time (US & Canada)


In [5]:
df = df.drop(columns=['tweet_location','retweet_count','name','airline_sentiment_gold','negativereason_gold','tweet_coord','negativereason','negativereason_confidence'])
df

,tweet_id,airline_sentiment,airline_sentiment_confidence,airline,text,tweet_created,user_timezone
0,570306133677760513,neutral,1.0000,Virgin America,@VirginAmerica What @dhepburn said.,2015-02-24 11:35:52 -0800,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,Virgin America,@VirginAmerica plus you've added commercials t...,2015-02-24 11:15:59 -0800,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,Virgin America,@VirginAmerica I didn't today... Must mean I n...,2015-02-24 11:15:48 -0800,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Virgin America,@VirginAmerica it's really aggressive to blast...,2015-02-24 11:15:36 -0800,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Virgin America,@VirginAmerica and it's a really big bad thing...,2015-02-24 11:14:45 -0800,Pacific Time (US & Canada)
...,...,...,...,...,...,...,...
14635,569587686496825344,positive,0.3487,American,@AmericanAir thank you we got on a different f...,2015-02-22 12:01:01 -0800,NaN
14636,569587371693355008,negative,1.0000,American,@AmericanAir leaving over 20 minutes Late Flig...,2015-02-22 11:59:46 -0800,NaN
14637,569587242672398336,neutral,1.0000,American,@AmericanAir Please bring American Airlines to...,2015-02-22 11:59:15 -0800,NaN
14638,569587188687634433,negative,1.0000,American,"@AmericanAir you have my money, you change my ...",2015-02-22 11:59:02 -0800,Eastern Time (US & Canada)


In [6]:
df = df.dropna(subset=['text'])
df

,tweet_id,airline_sentiment,airline_sentiment_confidence,airline,text,tweet_created,user_timezone
0,570306133677760513,neutral,1.0000,Virgin America,@VirginAmerica What @dhepburn said.,2015-02-24 11:35:52 -0800,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,Virgin America,@VirginAmerica plus you've added commercials t...,2015-02-24 11:15:59 -0800,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,Virgin America,@VirginAmerica I didn't today... Must mean I n...,2015-02-24 11:15:48 -0800,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Virgin America,@VirginAmerica it's really aggressive to blast...,2015-02-24 11:15:36 -0800,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Virgin America,@VirginAmerica and it's a really big bad thing...,2015-02-24 11:14:45 -0800,Pacific Time (US & Canada)
...,...,...,...,...,...,...,...
14635,569587686496825344,positive,0.3487,American,@AmericanAir thank you we got on a different f...,2015-02-22 12:01:01 -0800,NaN
14636,569587371693355008,negative,1.0000,American,@AmericanAir leaving over 20 minutes Late Flig...,2015-02-22 11:59:46 -0800,NaN
14637,569587242672398336,neutral,1.0000,American,@AmericanAir Please bring American Airlines to...,2015-02-22 11:59:15 -0800,NaN
14638,569587188687634433,negative,1.0000,American,"@AmericanAir you have my money, you change my ...",2015-02-22 11:59:02 -0800,Eastern Time (US & Canada)


In [7]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,airline,text,tweet_created,user_timezone
0,570306133677760513,neutral,1.0000,Virgin America,@VirginAmerica What @dhepburn said.,2015-02-24 11:35:52 -0800,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,Virgin America,@VirginAmerica plus you've added commercials t...,2015-02-24 11:15:59 -0800,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,Virgin America,@VirginAmerica I didn't today... Must mean I n...,2015-02-24 11:15:48 -0800,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Virgin America,@VirginAmerica it's really aggressive to blast...,2015-02-24 11:15:36 -0800,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Virgin America,@VirginAmerica and it's a really big bad thing...,2015-02-24 11:14:45 -0800,Pacific Time (US & Canada)


In [8]:
df.groupby(by=['airline']).count()

,tweet_id,airline_sentiment,airline_sentiment_confidence,text,tweet_created,user_timezone
airline,,,,,,
American,2759,2759,2759,2759,2759,1791
Delta,2222,2222,2222,2222,2222,1613
Southwest,2420,2420,2420,2420,2420,1659
US Airways,2913,2913,2913,2913,2913,1851
United,3822,3822,3822,3822,3822,2527
Virgin America,504,504,504,504,504,379


In [9]:
text = list(df['text'])
text

['@VirginAmerica What @dhepburn said.',
 "@VirginAmerica plus you've added commercials to the experience... tacky.",
 "@VirginAmerica I didn't today... Must mean I need to take another trip!",
 '@VirginAmerica it\'s really aggressive to blast obnoxious "entertainment" in your guests\' faces &amp; they have little recourse',
 "@VirginAmerica and it's a really big bad thing about it",
 "@VirginAmerica seriously would pay $30 a flight for seats that didn't have this playing.\nit's really the only bad thing about flying VA",
 '@VirginAmerica yes, nearly every time I fly VX this “ear worm” won’t go away :)',
 '@VirginAmerica Really missed a prime opportunity for Men Without Hats parody, there. https://t.co/mWpG7grEZP',
 "@virginamerica Well, I didn't…but NOW I DO! :-D",
 "@VirginAmerica it was amazing, and arrived an hour early. You're too good to me.",
 '@VirginAmerica did you know that suicide is the second leading cause of death among teens 10-24',
 '@VirginAmerica I &lt;3 pretty graphic

In [10]:
def clean_text(text):
    text = re.sub(r'\W', ' ', str(text))
    text = text.lower()
    text = re.sub(r'\s+[a-z]\s+', ' ',text)
    text = re.sub(r'^[a-z]\s+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

In [11]:
df['text'] = df['text'].apply(clean_text)

In [12]:
stop_words = set(stopwords.words('english'))
df['text'] = df['text'].apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))

In [13]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,airline,text,tweet_created,user_timezone
0,570306133677760513,neutral,1.0000,Virgin America,virginamerica dhepburn said,2015-02-24 11:35:52 -0800,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,Virgin America,virginamerica plus added commercials experienc...,2015-02-24 11:15:59 -0800,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,Virgin America,virginamerica today must mean need take anothe...,2015-02-24 11:15:48 -0800,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Virgin America,virginamerica really aggressive blast obnoxiou...,2015-02-24 11:15:36 -0800,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Virgin America,virginamerica really big bad thing,2015-02-24 11:14:45 -0800,Pacific Time (US & Canada)


In [14]:
text = list(df['text'])
text

['virginamerica dhepburn said',
 'virginamerica plus added commercials experience tacky',
 'virginamerica today must mean need take another trip',
 'virginamerica really aggressive blast obnoxious entertainment guests faces amp little recourse',
 'virginamerica really big bad thing',
 'virginamerica seriously would pay 30 flight seats playing really bad thing flying va',
 'virginamerica yes nearly every time fly vx ear worm go away',
 'virginamerica really missed prime opportunity men without hats parody https co mwpg7grezp',
 'virginamerica well',
 'virginamerica amazing arrived hour early good',
 'virginamerica know suicide second leading cause death among teens 10 24',
 'virginamerica lt 3 pretty graphics much better minimal iconography',
 'virginamerica great deal already thinking 2nd trip australia amp even gone 1st trip yet p',
 'virginamerica virginmedia flying fabulous seductive skies take stress away travel http co ahlxhhkiyn',
 'virginamerica thanks',
 'virginamerica sfo pdx 

In [15]:
df['tweet_created'] = pd.to_datetime(df['tweet_created'], format='%Y-%m-%d %H:%M:%S %z')
df['date'] = df['tweet_created'].dt.strftime('%m, %d, %Y')
df['time'] = df['tweet_created'].dt.strftime('%H:%M')
df

,tweet_id,airline_sentiment,airline_sentiment_confidence,airline,text,tweet_created,user_timezone,date,time
0,570306133677760513,neutral,1.0000,Virgin America,virginamerica dhepburn said,2015-02-24 11:35:52-08:00,Eastern Time (US & Canada),"02, 24, 2015",11:35
1,570301130888122368,positive,0.3486,Virgin America,virginamerica plus added commercials experienc...,2015-02-24 11:15:59-08:00,Pacific Time (US & Canada),"02, 24, 2015",11:15
2,570301083672813571,neutral,0.6837,Virgin America,virginamerica today must mean need take anothe...,2015-02-24 11:15:48-08:00,Central Time (US & Canada),"02, 24, 2015",11:15
3,570301031407624196,negative,1.0000,Virgin America,virginamerica really aggressive blast obnoxiou...,2015-02-24 11:15:36-08:00,Pacific Time (US & Canada),"02, 24, 2015",11:15
4,570300817074462722,negative,1.0000,Virgin America,virginamerica really big bad thing,2015-02-24 11:14:45-08:00,Pacific Time (US & Canada),"02, 24, 2015",11:14
...,...,...,...,...,...,...,...,...,...
14635,569587686496825344,positive,0.3487,American,americanair thank got different flight chicago,2015-02-22 12:01:01-08:00,NaN,"02, 22, 2015",12:01
14636,569587371693355008,negative,1.0000,American,americanair leaving 20 minutes late flight war...,2015-02-22 11:59:46-08:00,NaN,"02, 22, 2015",11:59
14637,569587242672398336,neutral,1.0000,American,americanair please bring american airlines bla...,2015-02-22 11:59:15-08:00,NaN,"02, 22, 2015",11:59
14638,569587188687634433,negative,1.0000,American,americanair money change flight answer phones ...,2015-02-22 11:59:02-08:00,Eastern Time (US & Canada),"02, 22, 2015",11:59


In [16]:
df.drop(columns=['tweet_created'])

,tweet_id,airline_sentiment,airline_sentiment_confidence,airline,text,user_timezone,date,time
0,570306133677760513,neutral,1.0000,Virgin America,virginamerica dhepburn said,Eastern Time (US & Canada),"02, 24, 2015",11:35
1,570301130888122368,positive,0.3486,Virgin America,virginamerica plus added commercials experienc...,Pacific Time (US & Canada),"02, 24, 2015",11:15
2,570301083672813571,neutral,0.6837,Virgin America,virginamerica today must mean need take anothe...,Central Time (US & Canada),"02, 24, 2015",11:15
3,570301031407624196,negative,1.0000,Virgin America,virginamerica really aggressive blast obnoxiou...,Pacific Time (US & Canada),"02, 24, 2015",11:15
4,570300817074462722,negative,1.0000,Virgin America,virginamerica really big bad thing,Pacific Time (US & Canada),"02, 24, 2015",11:14
...,...,...,...,...,...,...,...,...
14635,569587686496825344,positive,0.3487,American,americanair thank got different flight chicago,NaN,"02, 22, 2015",12:01
14636,569587371693355008,negative,1.0000,American,americanair leaving 20 minutes late flight war...,NaN,"02, 22, 2015",11:59
14637,569587242672398336,neutral,1.0000,American,americanair please bring american airlines bla...,NaN,"02, 22, 2015",11:59
14638,569587188687634433,negative,1.0000,American,americanair money change flight answer phones ...,Eastern Time (US & Canada),"02, 22, 2015",11:59


# Results

You may have done tons of work on this. Not all of it belongs here. 

Reports should have a __narrative__. Once you've looked through all your results over the quarter, decide on one main point and 2-4 secondary points you want us to understand. Include the detailed code and analysis results of those points only; you should spend more time/code/plots on your main point than the others.

If you went down any blind alleys that you later decided to not pursue, please don't abuse the TAs time by throwing in 81 lines of code and 4 plots related to something you actually abandoned.  Consider deleting things that are not important to your narrative.  If its slightly relevant to the narrative or you just want us to know you tried something, you could keep it in by summarizing the result in this report in a sentence or two, moving the actual analysis to another file in your repo, and providing us a link to that file.

### Subsection 1

You will likely have different subsections as you go through your report. For instance you might start with an analysis of the dataset/problem and from there you might be able to draw out the kinds of algorithms that are / aren't appropriate to tackle the solution.  Or something else completely if this isn't the way your project works.

### Subsection 2

Another likely section is if you are doing any feature selection through cross-validation or hand-design/validation of features/transformations of the data

### Subsection 3

Probably you need to describe the base model and demonstrate its performance.  Maybe you include a learning curve to show whether you have enough data to do train/validate/test split or have to go to k-folds or LOOCV or ???

### Subsection 4

Perhaps some exploration of the model selection (hyper-parameters) or algorithm selection task. Validation curves, plots showing the variability of perfromance across folds of the cross-validation, etc. If you're doing one, the outcome of the null hypothesis test or parsimony principle check to show how you are selecting the best model.

### Subsection 5 

Maybe you do model selection again, but using a different kind of metric than before?



# Discussion

### Interpreting the result

OK, you've given us quite a bit of tech informaiton above, now its time to tell us what to pay attention to in all that.  Think clearly about your results, decide on one main point and 2-4 secondary points you want us to understand. Highlight HOW your results support those points.  You probably want 2-5 sentences per point.

### Limitations

Are there any problems with the work?  For instance would more data change the nature of the problem? Would it be good to explore more hyperparams than you had time for?   

### Ethics & Privacy

If your project has obvious potential concerns with ethics or data privacy discuss that here.  Almost every ML project put into production can have ethical implications if you use your imagination. Use your imagination.

Even if you can't come up with an obvious ethical concern that should be addressed, you should know that a large number of ML projects that go into producation have unintended consequences and ethical problems once in production. How will your team address these issues?

Consider a tool to help you address the potential issues such as https://deon.drivendata.org

### Conclusion

Reiterate your main point and in just a few sentences tell us how your results support it. Mention how this work would fit in the background/context of other work in this field if you can. Suggest directions for future work if you want to.

# Footnotes
<a name="lorenznote"></a>1.[^](#lorenz): Lorenz, T. (9 Dec 2021) Birds Aren’t Real, or Are They? Inside a Gen Z Conspiracy Theory. *The New York Times*. https://www.nytimes.com/2021/12/09/technology/birds-arent-real-gen-z-misinformation.html<br> 
<a name="admonishnote"></a>2.[^](#admonish): Also refs should be important to the background, not some randomly chosen vaguely related stuff. Include a web link if possible in refs as above.<br>
<a name="sotanote"></a>3.[^](#sota): Perhaps the current state of the art solution such as you see on [Papers with code](https://paperswithcode.com/sota). Or maybe not SOTA, but rather a standard textbook/Kaggle solution to this kind of problem
